In [ ]:
import numpy as np
import pandas as pd
from climateUtils import eca_read_station_like_file
import matplotlib.pyplot as plt

In [ ]:
stations_pres = eca_read_station_like_file("eca_data/ECA_blend_station_pp.txt")
stations_prec = eca_read_station_like_file("eca_data/ECA_blend_station_rr.txt")
stations_tavg = eca_read_station_like_file("eca_data/ECA_blend_station_tg.txt")
stations_rads = eca_read_station_like_file("eca_data/ECA_blend_station_qq.txt")

stations_dict = {'pp' : stations_pres, 'rr' : stations_prec, 'tg' : stations_tavg, 'qq' : stations_rads}


stations = pd.concat([stations_pres, stations_prec,stations_tavg,stations_rads ]).drop_duplicates().reset_index(drop=True)

In [ ]:
stations_pres

In [ ]:
dataframes = []
for measure, stations in stations_dict.items():
    for _,row in stations.iterrows():
        if(_ % 500 == 0):
            print(_)
        stat_id = row['STAID']
        filename = "eca_data/ECA_blend_%s/%s_STAID%06i.txt" % (measure, measure.upper(),stat_id)
        df = eca_read_station_like_file(filename)
        df['DATE'] = pd.to_datetime(df['DATE'], format="%Y%m%d")
        df[df == -9999] = np.nan
        df.dropna(axis=0, inplace=True)
        df = df[['STAID', 'DATE', measure.upper()]]
        df.rename({'STAID': 'station_id', 'DATE' : 'date', measure.upper() : 'value' }, axis=1, inplace=True)
        df['measure'] = measure
        df.set_index(['station_id', 'measure', 'date'], drop=True, inplace=True)
        dataframes.append(df)
dataframes = pd.concat(dataframes, axis=0)

In [ ]:
measure_x = 'pp'
measure_y = 'tg'
subset = pd.merge(dataframes.xs(measure_x, level='measure'), dataframes.xs(measure_y, level='measure'),left_index=True, right_index=True, how='inner')

In [ ]:
stations_df = stations_dict[measure_x]

In [ ]:
mutual_variance = {}
mutual_info = {}
mutual_fano = {}
for station_id, group in subset.groupby('station_id'):
    group_indexed = group.reset_index(level='station_id', drop=True)
    
    rolling_cov = group_indexed['value_x'].rolling('30D').cov(group_indexed['value_y'])
    rolling_varx = group_indexed['value_x'].rolling('30D').var()
    rolling_vary = group_indexed['value_y'].rolling('30D').var()
    
    mutual_info[station_id] = 0.5*np.log((rolling_varx*rolling_vary)/(rolling_varx*rolling_vary-rolling_cov*rolling_cov))
    mutual_variance[station_id] = 1- np.exp(-2*mutual_info[station_id] ) #only for 2 vars
    mutual_fano[station_id] = mutual_variance[station_id]/mutual_info[station_id]

In [ ]:
date = np.datetime64('2010-01-01')


In [ ]:
for counter,date in enumerate(np.arange(np.datetime64('2010-01-01'), np.datetime64('2015-12-31'))):
    data_plot = []
    for stat_id in mutual_fano.keys():
        dioporco = mutual_info[stat_id]
        location = stations_df[stations_df['STAID'] == stat_id][['LAT','LON']].values.flatten()
        values = dioporco.values[dioporco.index==date]
        
        if(len(values) == 0):
            continue
        data_plot.append((location[0], location[1], values[0]))
        
        
    data_plot = np.array(data_plot)
    fig, ax = plt.subplots(figsize=(10,6))
    plt.scatter(data_plot[:,1], data_plot[:,0], c=data_plot[:,2], s=3, vmin=0.0, vmax=0.8)
    ax.set_aspect('equal')
    plt.savefig("plots/%i.png" % counter)
    plt.close()